In [1]:
import torch
import yaml

from models.autoencoder import Autoencoder
from data.era5 import ERA5Dataset, gen_bgen

In [2]:
with open("configs/autoencoder_kl_f16_attention.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

device = "cpu"

# model = Autoencoder.load_from_checkpoint(
#     "checkpoints/vae-kl-f16-1440x720-69c-attention--step=31000.ckpt", config=config, strict=False).to(device)

model = Autoencoder(config).to(device)

model.train()

Autoencoder(
  (encoder): Encoder(
    (conv_in): Conv2d(69, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down_blocks): ModuleList(
      (0): DownBlock(
        (layers): ModuleList(
          (0): Sequential(
            (0): GroupNorm(32, 128, eps=1e-05, affine=True)
            (1): SiLU()
            (2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (3): GroupNorm(32, 256, eps=1e-05, affine=True)
            (4): SiLU()
            (5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )
          (1): Sequential(
            (0): GroupNorm(32, 256, eps=1e-05, affine=True)
            (1): SiLU()
            (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (3): GroupNorm(32, 256, eps=1e-05, affine=True)
            (4): SiLU()
            (5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )
        )
        (residual_input_conv): Modul

In [3]:
bgen = gen_bgen(train=False)
dataset = ERA5Dataset(bgen)

sample = dataset[100].unsqueeze(0)

In [4]:
sample = sample.to(model.device)

rec, posterior = model.forward(sample)

In [5]:
_, ae_log = model.loss_model.forward(
    inputs=sample,
    reconstructions=rec,
    posteriors=posterior,
    global_step=20000,
    optimizer="Generator",
    last_layer=model.get_last_layer()
)

tensor(3.5265)
tensor(3.5265)
tensor(1.7633)


In [6]:
ae_log

{'total_loss': tensor(0.9351),
 'kl_loss': tensor(0.0107),
 'rec_loss': tensor(0.9100),
 'd_weight': tensor(1.7633),
 'disc_factor': tensor(1.),
 'g_loss': tensor(0.0081)}